In [1]:
import yfinance as yf
import talib as ta
import pandas as pd
from datetime import datetime
from sklearn.preprocessing import StandardScaler

In [ ]:
import pandas as pd
import mlflow

df = pd.read_csv("./data/sp500_predict_2023-11-15.csv")
df.set_index('Date', inplace=True)

DEFAULT_MODEL = "runs:/c413fe91b94f4a8db7492ffa6657a0f6/model"
loaded_model = mlflow.pyfunc.load_model(DEFAULT_MODEL)
prediction = loaded_model.predict(df)

print(prediction)

In [2]:
def save_to_influx(data:dict):
    from influxdb_client_3 import InfluxDBClient3, Point
    token = "nlg1Z5pTNYCdwYwK7jh1LV6lIoJAYnYdLpgZNNqBdLrnX4d8PnBNTJfBDNhtrc9_tzGiBLyJE4Zh4pEP5I45VQ=="
    org = "Dev Team"
    host = "https://us-east-1-1.aws.cloud2.influxdata.com"
    bucket = "sp500_bucket"
    
    client = InfluxDBClient3(host=host, 
                             token=token, 
                             org=org)
    
    # os.environ['INFLUXDB_HOST']
    # os.environ['INFLUXDB_ORG']
    # os.environ['INFLUXDB_TOKEN']
    
    point = Point("stock_data")
    for key, value in data.items():
        point = point.field(key, value)
    
    client.write(database=bucket, record=point)
    client.close()
    print("Complete. Return to the InfluxDB UI.")

In [3]:
def get_data(stock_wkn="^GSPC"):
    # Install TaLIb:      https://cloudstrata.io/install-ta-lib-on-ubuntu-server/
    sp500_data = yf.download(stock_wkn, start="1980-01-01", end=datetime.now().strftime('%Y-%m-%d'))
    inputs = sp500_data['Close']
    time_period = 10
    sp500_data['Target'] = (sp500_data['Close'].shift(-10) > sp500_data['Close']).astype(int)
    sp500_data['Rendite'] = sp500_data['Close'].pct_change()
    sp500_data = sp500_data.dropna()
    sp500_data[f"SMA {time_period}"]      = ta.SMA(inputs, timeperiod = time_period)
    sp500_data[f"EMA {time_period}"]      = ta.EMA(inputs, timeperiod = time_period)
    sp500_data[f"EMA {20}"]               = ta.EMA(inputs, timeperiod = 20)
    sp500_data[f"WMA {time_period}"]      = ta.WMA(inputs, timeperiod = time_period)
    sp500_data[f"Momentum {time_period}"] = ta.MOM(inputs, timeperiod = time_period)
    sp500_data["SAR"] = ta.SAR(sp500_data["High"], sp500_data["Low"], acceleration=0.02, maximum=0.2)
    sp500_data["RSI"] = ta.RSI(sp500_data["Close"], timeperiod = 14)
    sp500_data["ROC"] = ta.ROC(sp500_data["Close"], timeperiod = 10) # On-Balance-Volume
    sp500_data["%R"]  = ta.WILLR(sp500_data["High"], sp500_data["Low"], sp500_data["Close"], timeperiod = 14)
    sp500_data["OBV"] = ta.OBV(sp500_data["Close"], sp500_data["Volume"])
    sp500_data["MACD"], sp500_data["MACD_SIGNAL"], sp500_data["MACD_HIST"] = ta.MACD(sp500_data["Close"], fastperiod=12, slowperiod=26, signalperiod=9)
    sp500_data["CCI"] = ta.CCI(sp500_data["High"], sp500_data["Low"], sp500_data["Close"], timeperiod = 14)
    sp500_data["ADOSC"] = ta.ADOSC(sp500_data["High"], sp500_data["Low"], sp500_data["Close"], sp500_data["Volume"], fastperiod=3, slowperiod=10)
    sp500_data["%K"] = (sp500_data['Close'] - sp500_data['Low']) * 100 / (sp500_data['High'] - sp500_data['Low'])
    sp500_data["%D"] =  sp500_data['%K'].rolling(3).mean()
    sp500_data = sp500_data.dropna() #print("Anzahl der Zeilen mit mindestens einem NaN-Wert:", sp500_data.isna().any(axis=1).sum())
    sp500_data.drop(["High", "Low", "Adj Close", "Open"], axis=1, inplace=True)
    
    # Scaling
    numeric_columns = sp500_data.select_dtypes(include=['float64', 'int64']).columns
    scaler = StandardScaler()
    sp500_data[numeric_columns] = scaler.fit_transform(sp500_data[numeric_columns])
    target_column = sp500_data.pop('Target')
    sp500_data['Target'] = target_column

    # Filter last day 
    yesterday_df = sp500_data.drop("Target", axis=1)
    yesterday_df = yesterday_df.tail(1)
    last_day = yesterday_df.index[0].strftime('%Y-%m-%d')
    yesterday_df.to_csv(f'./data/sp500_predict_{last_day}.csv', index=True)
    
    # Save last day to influx
    yesterday_data = yesterday_df.iloc[0].to_dict()
    yesterday_data['Date'] = yesterday_df.index[0].isoformat()
    save_to_influx(data=yesterday_data)
    
    # Save trainings data
    sp500_data = sp500_data.iloc[:-10] # Drop last 10 days
    sp500_data.to_csv(f'./data/sp500_training_data_to_{last_day}.csv', index=True)
    
    return sp500_data, yesterday_df

In [4]:
df, yesterday_df = get_data()

[*********************100%%**********************]  1 of 1 completed


/tmp/ipykernel_42929/2466630639.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sp500_data[f"SMA {time_period}"]      = ta.SMA(inputs, timeperiod = time_period)
/tmp/ipykernel_42929/2466630639.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sp500_data[f"EMA {time_period}"]      = ta.EMA(inputs, timeperiod = time_period)
/tmp/ipykernel_42929/2466630639.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value in

Complete. Return to the InfluxDB UI.


In [5]:
def get_data_form_influx():
    from influxdb_client_3 import flight_client_options, InfluxDBClient3
    import certifi
    fh = open(certifi.where(), "r")
    cert = fh.read()
    fh.close()

    token = "nlg1Z5pTNYCdwYwK7jh1LV6lIoJAYnYdLpgZNNqBdLrnX4d8PnBNTJfBDNhtrc9_tzGiBLyJE4Zh4pEP5I45VQ=="
    org = "Dev Team"
    host = "https://us-east-1-1.aws.cloud2.influxdata.com"
    bucket = "sp500_bucket"
    # os.environ['INFLUXDB_HOST']
    # os.environ['INFLUXDB_ORG']
    # os.environ['INFLUXDB_TOKEN']
    client = InfluxDBClient3(host=host, 
                                 token=token, 
                                 org=org, 
                                 flight_client_options=flight_client_options(tls_root_certs=cert),
                                 database=bucket)
    
    query = """SELECT *
    FROM "stock_data"
    WHERE time > now() - 24h"""
    
    
    result = client.query(query=query, database=bucket, language="influxql")
    
    df = result.to_pandas().drop(["iox::measurement", "time"], axis=1)
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)
    return df

In [6]:
get_data_form_influx()

,%D,%K,%R,ADOSC,CCI,Close,EMA 10,EMA 20,MACD,MACD_HIST,MACD_SIGNAL,Momentum 10,OBV,ROC,RSI,Rendite,SAR,SMA 10,Volume,WMA 10
Date,,,,,,,,,,,,,,,,,,,,
2023-11-15,0.019175,-0.784919,1.133682,1.40634,0.992047,2.924371,2.836735,2.802099,2.04021,4.952725,0.477527,4.766703,2.132107,1.824033,1.229881,0.105997,2.753382,2.835659,1.275609,2.857765
2023-11-15,0.019175,-0.784919,1.133682,1.40634,0.992047,2.924371,2.836735,2.802099,2.04021,4.952725,0.477527,4.766703,2.132107,1.824033,1.229881,0.105997,2.753382,2.835659,1.275609,2.857765
2023-11-15,0.019175,-0.784919,1.133682,1.40634,0.992047,2.924371,2.836735,2.802099,2.04021,4.952725,0.477527,4.766703,2.132107,1.824033,1.229881,0.105997,2.753382,2.835659,1.275609,2.857765
2023-11-15,0.019175,-0.784919,1.133682,1.40634,0.992047,2.924371,2.836735,2.802099,2.04021,4.952725,0.477527,4.766703,2.132107,1.824033,1.229881,0.105997,2.753382,2.835659,1.275609,2.857765
2023-11-15,0.019175,-0.784919,1.133682,1.40634,0.992047,2.924371,2.836735,2.802099,2.04021,4.952725,0.477527,4.766703,2.132107,1.824033,1.229881,0.105997,2.753382,2.835659,1.275609,2.857765
